In [1]:
import pathlib

import pandas as pd

In [2]:
file_path = pathlib.Path(
    "../../data/5.converted_data/mean_aggregated_data.parquet"
).resolve(strict=True)
df = pd.read_parquet(file_path)
df.head()

,Metadata_genotype,Metadata_replicate,AreaShape_Area,AreaShape_CentralMoment_0_0,AreaShape_CentralMoment_0_1,AreaShape_CentralMoment_0_2,AreaShape_CentralMoment_0_3,AreaShape_CentralMoment_1_0,AreaShape_CentralMoment_1_1,AreaShape_CentralMoment_1_2,...,Texture_SumEntropy_OP_3_02_256,Texture_SumEntropy_OP_3_03_256,Texture_SumVariance_OP_3_00_256,Texture_SumVariance_OP_3_01_256,Texture_SumVariance_OP_3_02_256,Texture_SumVariance_OP_3_03_256,Texture_Variance_OP_3_00_256,Texture_Variance_OP_3_01_256,Texture_Variance_OP_3_02_256,Texture_Variance_OP_3_03_256
0,high,1,0.596483,0.596483,0.270876,-0.049237,0.390868,2.100915,-0.463937,-0.042982,...,-0.104105,-0.119036,-0.545856,-0.525027,-0.543305,-0.544589,-0.557563,-0.562101,-0.558976,-0.560872
1,high,10,1.238954,1.238954,0.659813,1.722364,-0.874297,0.179195,-1.644760,0.825270,...,0.646977,0.635356,0.630682,0.642091,0.593754,0.635274,0.557863,0.548473,0.560985,0.547769
2,high,11,2.131207,2.131207,0.630866,1.426469,0.291591,1.801207,-0.270662,-0.164307,...,0.686771,0.678230,0.552382,0.570823,0.526266,0.548475,0.498825,0.486269,0.500083,0.490213
3,high,12,1.201238,1.201238,0.024943,1.279119,-0.464086,-1.652853,-0.982309,0.107680,...,0.309505,0.308933,-0.054779,-0.059876,-0.098400,-0.067930,-0.103782,-0.109367,-0.101086,-0.106772
4,high,13,1.194888,1.194888,-0.554368,-0.034466,0.050882,-1.344980,-0.483673,-0.794434,...,0.056706,0.047741,-0.325171,-0.295674,-0.321039,-0.322436,-0.348409,-0.356255,-0.350286,-0.352168


We want to remove CellProfiler features that do not accurately contribute to the shape variance.


Spatial moment features are removed as they are not invariant to rotation and translation.

Central moment features are removed as measurements of size is not of interest.

Normalized moment features are removed as they do not contribute to the shape variance.

EulerNumber is removed as we are not concerned with the number of holes in the shape.

Extent is removed as we do not care about the ratio of pixels in the bounding box to the pixels in the shape.

Orientation is removed as we are not concerned with the angle of the major axis of the shape.

Zernike features are removed as they are not invariant to rotation and translation.

Texture features are removed as we are not interested in the texture of the shape.

In [3]:
list_to_drop = [
    "SpatialMoment",
    "CentralMoment",
    "NormalizedMoment",
    "EulerNumber",
    "Extent",
    "Orientation",
    "Zernike",
    "Texture",
]

In [4]:
# drop columns that contain Metadata
df1 = df.copy()
df1 = df1.drop(columns=[col for col in df1.columns if "Metadata" in col])
columns = df1.columns.to_list()
print(df1.shape)
# if column contains any of the strings in list_to_drop, drop it
for col in list_to_drop:
    columns = [c for c in columns if col not in c]
df1 = df1[columns]
print(df1.shape)
df1["Metadata_genotype"] = df["Metadata_genotype"]
df1["Metadata_replicate"] = df["Metadata_replicate"]

(42, 244)
(42, 64)


In [5]:
df1.head()

,AreaShape_Area,AreaShape_Compactness,AreaShape_ConvexArea,AreaShape_Eccentricity,AreaShape_EquivalentDiameter,AreaShape_FormFactor,AreaShape_HuMoment_0,AreaShape_HuMoment_1,AreaShape_HuMoment_2,AreaShape_HuMoment_3,...,RadialDistribution_MeanFrac_OP_1of4,RadialDistribution_MeanFrac_OP_2of4,RadialDistribution_MeanFrac_OP_3of4,RadialDistribution_MeanFrac_OP_4of4,RadialDistribution_RadialCV_OP_1of4,RadialDistribution_RadialCV_OP_2of4,RadialDistribution_RadialCV_OP_3of4,RadialDistribution_RadialCV_OP_4of4,Metadata_genotype,Metadata_replicate
0,0.596483,0.461635,0.872478,-2.085548,0.674228,-0.003732,-0.719444,-0.669940,-0.802052,-0.642703,...,0.104420,-0.038983,-0.102821,0.180570,-0.829924,-0.748209,-0.367368,-0.447147,high,1
1,1.238954,-1.084807,1.140365,-0.097805,1.248412,1.278626,-0.765234,-0.608788,-0.922928,-0.641467,...,-1.248855,-1.375573,-1.299896,1.556094,0.891142,0.624669,1.105887,1.402458,high,10
2,2.131207,-0.755467,2.216982,-1.886368,1.891261,0.795809,-0.924378,-0.685352,-0.961462,-0.660985,...,0.071927,-0.521327,-0.895666,0.728454,0.602593,0.579514,0.529953,0.684432,high,11
3,1.201238,-1.171680,1.095314,-0.296606,1.213704,1.488535,-0.740447,-0.592887,-0.865995,-0.607569,...,-1.088251,-1.243669,-0.990843,1.272853,-0.454219,-0.089235,0.698538,0.836480,high,12
4,1.194888,-0.952083,1.100213,0.052727,1.197315,1.025243,-0.652542,-0.548787,-0.785373,-0.525058,...,-1.265077,-1.415683,-1.348218,1.679224,-0.579672,-0.938057,0.002271,0.626942,high,13


In [6]:
# save the dataframe
df_path = pathlib.Path(
    "../../data/5.converted_data/mean_aggregated_manual_feature_selected_data.parquet"
)
df1.to_parquet(df_path)